Original colab

In [ ]:
https://colab.research.google.com/drive/14pj2WaDH8gllQh3piJMoDtz6Vt5CXP7Z?usp=sharing

## Installing Necessary Libraries

In [ ]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.8/100.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 88.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7

# Dataset details
Instacart Data can be downloaded from [here](https://www.kaggle.com/competitions/instacart-market-basket-analysis/data). We just need product & department csv files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/'Colab Notebooks'


departments.csv  products.csv


In [ ]:
import pandas as pd
df = pd.read_csv("/content/midjourney_prompt_dataset.csv")

In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
train_df.head(10)

,User,text
93,"""midjourney prompt for a steampunk laboratory ...","""A blueprint of steampunk style Alchemist's La..."
66,midjourney prompt for a modern Tower of Babel,"Tower of Babel, reaching up to heaven, epic ci..."
63,"""midjourney prompt for a Viking warrior with a...","""Norse amulet made from black iron with a carb..."
79,midjourney prompt for a typography design with...,"Intricate typography design the words ""OXYDE N..."
25,"""midjourney prompt for a steampunk castle inte...","""A blueprint of steampunk style Disney Castle,..."
194,"""midjourney prompt for a fashion model with un...","""a realistic photo portrait of a single beauti..."
146,"""midjourney prompt for a vintage travel poster...","""1940s poster for a far off land with a river ..."
255,midjourney prompt for an Emirati prince riding...,Emirati prince looking directly at the camera ...
19,"""midjourney prompt for a cyberpunk street scen...","""Cyberpunk futuristic crowded sci-fi street As..."
280,midjourney prompt for a colorful and playful g...,"A semi-translucent gummy bear, digital art"


In [ ]:
test_df.head(10)

,User,text
45,midjourney prompt for a beautiful portrait of ...,a realistic photo portrait of a single beautif...
157,"""midjourney prompt for a battle Mecha in a fut...","""Large battle Mecha helping in the constructio..."
256,"""midjourney prompt for a cyberpunk backlit por...","""futuristic, backlit silhouette face of a pers..."
42,"""midjourney prompt for a chaotic and detailed ...","""Bill Sienkiewicz Boombox complex complicated ..."
181,midjourney prompt for a Keith Haring inspired ...,people are dancing painting in the style of Ke...
9,midjourney prompt for a Norse amulet with glow...,Norse amulet made from black iron with a carbo...
46,midjourney prompt for a dinosaur bone excavati...,A landscape of a Jurassic period jungle with g...
172,midjourney prompt for a Gothic church with a b...,A classical oil painting of the interior of a ...
77,"""midjourney prompt for a steampunk castle inte...","""A blueprint of steampunk style Disney Castle,..."
195,"""midjourney prompt for a modern Tower of Babel...","""Tower of Babel, reaching up to heaven, epic c..."


In [ ]:
from datasets import Dataset,DatasetDict
train_dataset_dict = DatasetDict({
    "train": Dataset.from_pandas(train_df),
})

## Loading the model

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "TinyPixel/Llama-2-7B-bf16-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

config.json:   0%|          | 0.00/626 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00014.bin:   0%|          | 0.00/981M [00:00<?, ?B/s]

pytorch_model-00002-of-00014.bin:   0%|          | 0.00/967M [00:00<?, ?B/s]

pytorch_model-00003-of-00014.bin:   0%|          | 0.00/967M [00:00<?, ?B/s]

...

pytorch_model-00012-of-00014.bin:   0%|          | 0.00/967M [00:00<?, ?B/s]

pytorch_model-00013-of-00014.bin:   0%|          | 0.00/967M [00:00<?, ?B/s]

pytorch_model-00014-of-00014.bin:   0%|          | 0.00/847M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Let's also load the tokenizer below

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

**Let's check what the base model predicts before finetuning. :)**

In [ ]:
import transformers
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)


sequences = pipeline(
   ["midjourney prompt for a boy running in the snow"],
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

# print(f"Result: {sequences[0]['generated_text']}")
for seq in sequences:
    print(f"Result: {seq[0]['generated_text']}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Result: midjourney prompt for a boy running in the snow, with some snowflakes. hopefully this is helpful to somebody, as I'm sure there is some other person out there who needs this, like how I needed a lot of things in the past.
#winter#boy#snow#snowflakes#midjourney#midjourney prompts#mjprompt#midjourney prompt#prompt
#115: "What does winter symbolize to you?"
This is the first time I've been asked this question by another student at school. I was pretty surprised because I don't think I had ever been asked that question before and I don't know why it caught me so off guard.
Winter to me is a time of reflection for many people. For some people, it's a time where they can reflect on what they've done wrong and make plans to


Below we will load the configuration file in order to create the LoRA model. According to QLoRA paper, it is important to consider all linear layers in the transformer block for maximum performance.

In [ ]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","v_proj"]
)

## Loading the trainer

Here we will use the [`SFTTrainer` from TRL library](https://huggingface.co/docs/trl/main/en/sft_trainer) that gives a wrapper around transformers `Trainer` to easily fine-tune models on instruction based datasets using PEFT adapters. Let's first load the training arguments below.

In [ ]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 1
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 120
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

Then finally pass everthing to the trainer

In [ ]:
from datasets import load_dataset

In [ ]:
data = load_dataset("csv", data_files="midjourney_prompt_dataset.csv")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
data["train"][0]

KeyError: ignored

In [ ]:
def generate_prompt(data_point):
  return f"""
<human>: {data_point["User"]}
<assistant>: {data_point["text"]}
""".strip()

def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

In [ ]:
data = data["train"].shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/288 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
data

Dataset({
    features: ['User', 'text', 'input_ids', 'attention_mask'],
    num_rows: 288
})

In [ ]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=data,
    # train_dataset=data['train'],
    peft_config=peft_config,
    dataset_text_field="text",
    # dataset_text_field="prediction",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


Map:   0%|          | 0/288 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:247: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


We will also pre-process the model by upcasting the layer norms in float 32 for more stable training

In [ ]:
model.config.use_cache = False

In [ ]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

## Train the model

Now let's train the model! Simply call `trainer.train()`

In [ ]:
import torch

# Clear GPU memory
torch.cuda.empty_cache()

wand username = cipatih546@cumzle.com
wand password == testtest12
wandb api = c5e33e66b29539c9cc4585743f29fbdabfebdba6

In [ ]:
trainer.train()

OutOfMemoryError: ignored

In [ ]:
lst_test_data = list(test_df['text'])

In [ ]:
len(lst_test_data)

9938

In [ ]:
sample_size = 25
lst_test_data_short = lst_test_data[:sample_size]

In [ ]:
import transformers

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # torch_dtype=torch.bfloat16,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
)

sequences = pipeline(
    lst_test_data_short,
    max_length=100,  #200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

for ix,seq in enumerate(sequences):
    print(ix,seq[0]['generated_text'])

0 Free & Clear Stage 4 Overnight Diapers ->: babies: diapers & accessories diapers ->: baby needs diapers & pads free clear diapers ->: baby needs diapers free & clear stage 4 overnight diapers: diapers ->: baby needs diapers diapers & accessories ->: baby needs diapers free & clear stage 4 overnight diapers: diapers ->: personal care baby care ->: diapers & accessories dia
1 Beef pot roast with roasted potatoes, carrots, sweet onions, green beans, and a rich gravy Beef Pot Roast ->: frozen goods ->: frozen meals ->: meat, chicken & seafood ->: beef & beef products ->: entree ->: entree ->: entree ->: entrees: beef ->: beef entree ->: meat ->: beef ->: meat, chicken
2 Coffee Liquer ->: alcoholic beverages spirits ->: beverages beer & hard cider ->: beer coffee liqueur ->: beverages liquor ->: beverages beer & hard cider coffee liquor ->: beverages beer coffee liqueur original ->: beverages liquor ->: beverages beer & hard cider coffee liqueur original alcoholic beverages spirits drinks

In [ ]:
def correct_answer(ans):
  return (ans.split("->:")[1]).strip()

answers = []
for ix,seq in enumerate(sequences):
    # print(ix,seq[0]['generated_text'])
    answers.append(correct_answer(seq[0]['generated_text']))

answers

['babies: diapers & accessories diapers',
 'frozen goods',
 'alcoholic beverages spirits',
 'bakery',
 'personal care soap & bath',
 'household: pantry: cleaning: broom & mop: broom & broom brushes: hand broom',
 'deli meats & cheese',
 'beverages alcohol juice cranberry raspberry juice cocktail',
 'dairy eggs butter & cheese eggs',
 'canned goods',
 'dairy eggs & dairy eggs',
 'snacks chocolate & candy',
 'dry goods pasta, rice, beans and grains pasta',
 'personal care toiletries personal care toiletries liquid hand wash & dish wash',
 'beverages',
 'international foods dry goods spices & seasonings spices five spices',
 'breakfast cereals',
 'dairy eggs & cheese frozen dairy',
 'meat seafood frozen',
 'household essentials candles & accessories scented votive wax tablets',
 'frozen food',
 'dairy eggs dairy egg products soy products',
 'breakfast foods',
 'personal care',
 'meat seafood fish seafood']

In [ ]:
df_evaluate = test_df.iloc[:sample_size][['product_name','department']]

df_evaluate = df_evaluate.reset_index(drop=True)

df_evaluate['department_predicted'] = answers

df_evaluate

,product_name,department,department_predicted
0,Free & Clear Stage 4 Overnight Diapers,babies,babies: diapers & accessories diapers
1,"Beef pot roast with roasted potatoes, carrots,...",frozen,frozen goods
2,Coffee Liquer,alcohol,alcoholic beverages spirits
3,Bread Rolls,bakery,bakery
4,French Milled Oval Almond Gourmande Soap,personal care,personal care soap & bath
5,Dust Pan,household,household: pantry: cleaning: broom & mop: broo...
6,Roasted Pine Nut Hommus,deli,deli meats & cheese
7,Cranberry Raspberry Juice Cocktail,beverages,beverages alcohol juice cranberry raspberry ju...
8,Sweet Cream Butter Salted,dairy eggs,dairy eggs butter & cheese eggs
9,Traditional Chicken Barley Soup,canned goods,canned goods
